Image data for this task can be dowloaded from here: https://github.com/ardamavi/Sign-Language-Digits-Dataset

In [ ]:
# Python 2.x
from __future__ import division

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from keras.utils import np_utils 
from keras.models import Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Input, Flatten
from keras.initializers import glorot_uniform
from keras import metrics

In [ ]:
FOLDER = "images/"
NR_CLASSES = 10

In [ ]:
class ImageReader(object):
    def __init__(self, folder, nrows=100, ncols=100):
        self._folder = folder
        self._nrows = nrows
        self._ncols = ncols
    
    def _read_images(self):
        labels = get_foldernames(self._folder)
        for label in labels:
            subfolder = self._folder + label + "/"
            image_names = get_filenames(subfolder)
            for filename in image_names:
                path = subfolder + filename
                image = np.array(Image.open(path).convert("L")) / 255
                yield image, label
            
    def read_data(self):
        images = []
        labels = []
        input_data = self._read_images()
        for image, label in input_data:
            if image.shape == (self._nrows, self._ncols):   
                images.append(image)
                labels.append(label)
        tensor = np.asarray(images)
        length, row, col = tensor.shape
        return tensor.reshape(length, row, col, 1), labels        
    

def get_foldernames(root):
    return sorted([folder for folder in os.listdir(root) if os.path.isdir(os.path.join(root, folder))])


def get_filenames(folder):
    return sorted([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])

In [ ]:
reader = ImageReader(FOLDER)

In [ ]:
X, y = reader.read_data()

In [ ]:
def print_image_for_each_label(X, y):
    def find_index_of_label(y, label):
        for ix, item in enumerate(y):
            if item == label:
                return ix
  
    fig = plt.figure(figsize=(16, 6))
    labels = sorted(list(set(y)))
    for p, label in enumerate(labels):
        ix = find_index_of_label(y, label)
        image = np.squeeze(X[ix, :])
        img = fig.add_subplot(2, 5, p+1)
        imgplot = plt.imshow(image, cmap='gray')
    plt.show()

In [ ]:
print_image_for_each_label(X, y)

In [ ]:
def show_image(matrix):
    fig = plt.figure(figsize=(4, 4))
    img = fig.add_subplot(1, 1, 1)
    imgplot = plt.imshow(np.squeeze(matrix), cmap='gray')
    plt.show()

In [ ]:
show_image(X[1000, :])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2112)

In [ ]:
Counter(y_test)

In [ ]:
m_train, image_size, _, _ = X_train.shape
m_test = X_test.shape[0]

In [ ]:
print("Number of training examples: {}".format(m_train))

In [ ]:
X_train_flattened = X_train.reshape(m_train, -1)
X_test_flattened = X_test.reshape(m_test, -1)

## Image classification with logistic regression

In [ ]:
model = LogisticRegression(random_state=2112, C=0.01, solver="newton-cg", multi_class="multinomial")

In [ ]:
model.fit(X_train_flattened, y_train)

In [ ]:
predicted = model.predict(X_test_flattened)

In [ ]:
well_classified = [ix for ix, (pred, ground) in enumerate(zip(predicted, y_test)) if pred == ground]
misclassified = [ix for ix, (pred, ground) in enumerate(zip(predicted, y_test)) if pred != ground]

In [ ]:
acc = model.score(X_test_flattened, y_test)

In [ ]:
print(acc)

In [ ]:
len(well_classified)

In [ ]:
len(misclassified)

In [ ]:
ix = well_classified[0]
show_image(X_test[ix])
print("label: {}".format(y_test[ix]))
print("prediction: {}".format(predicted[ix]))

In [ ]:
ix = misclassified[0]
show_image(X_test[ix])
print("label: {}".format(y_test[ix]))
print("prediction: {}".format(predicted[ix]))

In [ ]:
X_train.shape

## Image classification with a Convolutional Neural Network (CNN)

In [ ]:
Y_train = np_utils.to_categorical(y_train, NR_CLASSES) 
Y_test = np_utils.to_categorical(y_test, NR_CLASSES)

In [ ]:
def build_CNN(input_shape, nr_classes):
    X_input = Input(input_shape)
    X = Conv2D(filters=32, kernel_size=(7, 7), strides=(2, 2), kernel_initializer=glorot_uniform(seed=2112))(X_input)
    X = Activation("relu")(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = Conv2D(filters=16, kernel_size=(3, 3), strides=(2, 2))(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = Flatten()(X)
    X = Dense(nr_classes, activation="softmax")(X)
    model = Model(inputs=X_input, outputs=X)
    return model

In [ ]:
model = build_CNN((100, 100, 1), NR_CLASSES)

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=[metrics.categorical_accuracy])

In [ ]:
history = model.fit(x=X_train, y=Y_train, epochs=20, batch_size=16)

In [ ]:
predicted_probs = model.predict(X_test)
predicted_classes = np.argmax(predicted_probs, axis=1)

In [ ]:
predicted_classes

In [ ]:
_, acc = model.evaluate(X_test, Y_test)

In [ ]:
acc